In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import torch
import numpy as np 
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy

In [ ]:
#load features and labels
X =  np.load('Feature_Concatenation_MobileNet.npy')
X = X.reshape(X.shape[0],X.shape[2]*4)
y = np.load('Labels.npy')

#dataset split
X_train = X[:111]
X_val = X[111:-32]
X_test = X[-32:]
y_train = y[:111]
y_val = y[111:-32]
y_test = y[-32:]

In [ ]:
#example of training baseline model
clf1 = LogisticRegression(random_state=0).fit(X_train, y_train)
clf2 = GradientBoostingClassifier(random_state=0).fit(X_train, y_train)
clf3 = RandomForestClassifier(random_state=0).fit(X_train, y_train)
#Number of “Worse” versus “Improved” case in testing dataset:
N = y_test.size-(y_test==0).sum()
#Classification accuracy of discriminating “Worse” versus “Improved” case
print((clf1.predict(X_test)*y_test == 1).sum()/N)
print((clf2.predict(X_test)*y_test == 1).sum()/N)
print((clf3.predict(X_test)*y_test == 1).sum()/N)

In [ ]:
from torch.utils.data import Dataset, DataLoader

import numpy as np


class MyDataset(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).float()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
        
        return x, y
    
    def __len__(self):
        return len(self.data)

In [ ]:
#FC-classifier

In [ ]:
#load features and labels
X =  np.load('Feature_Concatenation_MobileNet.npy')
X = X.reshape(X.shape[0],X.shape[2]*4)
y = np.load('Labels.npy')
#reassign the label to positive
y[y<0]=2

#dataset split
X_train = X[:111]
X_val = X[111:-32]
X_test = X[-32:]
y_train = y[:111]
y_val = y[111:-32]
y_test = y[-32:]

In [ ]:
dataset_train = MyDataset(X_train,y_train)
dataset_val = MyDataset(X_val,y_val)
dataset_test = MyDataset(X_test,y_test)

In [ ]:
trainloader = DataLoader(
    dataset_train,
    batch_size=10,
    shuffle=True,
    num_workers=2,
    pin_memory=torch.cuda.is_available()
)
valloader = DataLoader(
    dataset_val,
    batch_size=5,
    shuffle=False,
    num_workers=2,
    pin_memory=torch.cuda.is_available()
)
testloader = DataLoader(
    dataset_test,
    batch_size=10,
    shuffle=False,
    num_workers=2,
    pin_memory=torch.cuda.is_available()
)

In [ ]:
feature_size = X.shape[1]

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(feature_size,300)
        self.fc2 = nn.Linear(300, 3)

    def forward(self, x):
        x = x.view(-1, feature_size)
        emb = F.relu(self.fc1(x))
        nn.Dropout(0.5), #50 % probability 
        x = self.fc2(emb)
        return x

In [ ]:
model = Net()
optimizer = optim.Adam(model.parameters(),lr=0.0001)
loss_function = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
num_epochs=100


for epoch in range(num_epochs):
   
    model.train()
    for inputs, labels in trainloader:
        outputs = model(inputs)
        loss = loss_function(outputs, labels)#+criterion1()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    results = []
    gt = []
    running_corrects = 0

    model.eval()
    results = []
    gt = []
    running_corrects = 0
    for inputs, labels in testloader:
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        results.append(preds.data.numpy())
        gt.append(labels.data.numpy())
    results = np.hstack(results)
    gt = np.hstack(gt)
    index = [gt!=0]
    print('test')
    print((results[index] == gt[index]).sum()/(gt!=0).sum())